**Simple demo to show ML Forecasting**
- Predict visitor/traffic volume based on weather forecast
- Knowing predicted volume can help with scheduling services such as police, parking, traffic, vendors, etc...



**What's Included**
- Creation of Historical Data - Traffic and Weather
- Creation of Forecast/Future Data
- Clickpath to Generate Model in AI & ML Studio
- Model Creation/Setup SQL
- Model Execution SQL
- Model Output Review SQL
 

**Creation of Historical Data - Traffic and Weather**

Marketplace weather data
- GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.HISTORY_DAY
- to get weather data - Snowsight > Data Products > Marketplace > Global Weather & Climate Data for BI
- this weather data does not include all postal_codes, if customizing verify your postal_code is in the dataset




In [ ]:
-- create weather table for same dates as above traffic and same zip code
CREATE or replace TABLE SNOWDEV.RAW.WEATHER AS 
select date_valid_std,POSTAL_CODE, MAX_TEMPERATURE_AIR_2M_F,
AVG_TEMPERATURE_AIR_2M_F,
TOT_PRECIPITATION_IN,
AVG_CLOUD_COVER_TOT_PCT
from GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.HISTORY_DAY where postal_code = '33163'  ;

In [ ]:
-- create a vehicle_traffic history table for our one targetted zip code
-- seeding with 12000
-- to do - incorate high/low season
-- the  result will be a table with one record per day and traffic info
-- this table will get joined back to weather data and will have our history

USE ROLE ACCOUNTADMIN;
SET  avg_visits_seed = 12000;
--- create traffic off weather table
CREATE or replace TABLE SNOWDEV.RAW.VEHICLE_TRAFFIC AS  
SELECT 
    date_valid_std AS TRAFFIC_DATE,
    '33163' as ZIP,
    $avg_visits_seed as average_visits_seed

    /*
    ,    
     CASE 
        WHEN EXTRACT(MONTH, date_valid_std) = 1 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 2 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 3 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 4 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 5 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 6 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 7 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 8 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 9 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 10 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 11 THEN 1
        WHEN EXTRACT(MONTH, date_valid_std) = 12 THEN 1
    END AS month_high_low_adjust
   */
    
    from GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.HISTORY_DAY where postal_code = '33163'  ;
 


In [ ]:
-- see records counts, should match
SELECT COUNT(*) as count, 'WEATHER' as table_name FROM SNOWDEV.RAW.WEATHER union 
SELECT COUNT(*), 'TRAFFIC' as table_name FROM  SNOWDEV.RAW.VEHICLE_TRAFFIC;

In [ ]:
-- preview traffic data
select * from SNOWDEV.RAW.VEHICLE_TRAFFIC limit 5;

In [ ]:
-- preview weather data
select * from SNOWDEV.RAW.WEATHER limit 5;

In [ ]:
-- Create a view that joins weather and traffic
-- case statements are used to adjust the visits number
-- so as weather changes (ex high precipitation) visits go down

CREATE or replace VIEW SNOWDEV.RAW.TRAFFIC_WEATHER_VW AS
SELECT 

    POSTAL_CODE, 
    MAX_TEMPERATURE_AIR_2M_F,
    AVG_TEMPERATURE_AIR_2M_F,
    TOT_PRECIPITATION_IN,
    AVG_CLOUD_COVER_TOT_PCT,
    AVERAGE_VISITS_SEED,
    --MONTH_HIGH_LOW_ADJUST,
    CASE 
            WHEN TOT_PRECIPITATION_IN > 4 THEN .2  
            WHEN TOT_PRECIPITATION_IN > 3 THEN .4  
            WHEN TOT_PRECIPITATION_IN > 2 THEN .6  
            WHEN TOT_PRECIPITATION_IN > 1 THEN .8  
            ELSE  1.2          
    END AS PRECIPITATION_ADJUST,
    CASE 
            WHEN MAX_TEMPERATURE_AIR_2M_F > 90 THEN .9
            ELSE  1.1            
    END AS MAX_TEMPERATURE_ADJUST,
    CASE 
            WHEN AVG_TEMPERATURE_AIR_2M_F > 85 THEN 1.1
            WHEN AVG_TEMPERATURE_AIR_2M_F > 80 THEN .9
            WHEN AVG_TEMPERATURE_AIR_2M_F > 70 THEN .75
            ELSE .7
    END AS AVG_TEMPERATURE_AIR_2M_F_ADJUST,
    uniform(-50, 51, random()) AS RANDOM_ADJUSTMENT,
ROUND((AVERAGE_VISITS_SEED  * PRECIPITATION_ADJUST * MAX_TEMPERATURE_ADJUST * AVG_TEMPERATURE_AIR_2M_F_ADJUST) + RANDOM_ADJUSTMENT) as VISITS,

   
   -- ROUND((AVERAGE_VISITS_SEED * MONTH_HIGH_LOW_ADJUST * PRECIPITATION_ADJUST * MAX_TEMPERATURE_ADJUST * AVG_TEMPERATURE_AIR_2M_F_ADJUST) + RANDOM_ADJUSTMENT) as VISITS,
   TRAFFIC_DATE
    
FROM 
    SNOWDEV.RAW.WEATHER WEA,
    SNOWDEV.RAW.VEHICLE_TRAFFIC TRAF
WHERE
    WEA.date_valid_std=TRAF.TRAFFIC_DATE
ORDER BY 
TRAFFIC_DATE;


In [ ]:
-- make a table of the view so you have static data that can be adjusted
--     to support for your demo story
-- keeping just the necessary columns to support the model, keeping it simple for now
create or replace table SNOWDEV.RAW.TRAFFIC_WEATHER_HISTORY as 
select 
TRAFFIC_DATE as date_valid_std,
POSTAL_CODE, 
MAX_TEMPERATURE_ADJUST * MAX_TEMPERATURE_AIR_2M_F  AS MAX_TEMPERATURE_AIR_2M_F,
AVG_TEMPERATURE_AIR_2M_F_ADJUST * AVG_TEMPERATURE_AIR_2M_F AS AVG_TEMPERATURE_AIR_2M_F,
PRECIPITATION_ADJUST * TOT_PRECIPITATION_IN as TOT_PRECIPITATION_IN,
VISITS
from 
SNOWDEV.RAW.TRAFFIC_WEATHER_VW ;

In [ ]:
select min(date_valid_std), max(date_valid_std) from SNOWDEV.RAW.TRAFFIC_WEATHER_HISTORY

**Generate Forecast/Future Data**

- A this point we have historical traffic data from 12/9/22 - 12/8/24
- Depending on when above statement were run you'll likely get different dates ranges
- Next step will be to get the weather forecast which will be the input for generating forecasted visits

In [ ]:
-- the forecast has 15 rows of future forecast data, one row per date for my zipcode
select * from GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.FORECAST_DAY where postal_code = '33163';


In [ ]:
-- create  a table so you have static data that can be adjusted
--     to support for your demo story
-- note to keep it simple I am only including the same columns that are in my history table
create or replace  table SNOWDEV.RAW.WEATHER_FORECAST AS 
select date_valid_std, postal_code, MAX_TEMPERATURE_AIR_2M_F, AVG_TEMPERATURE_AIR_2M_F, TOT_PRECIPITATION_IN from GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.FORECAST_DAY where postal_code = '33163';

In [ ]:
-- modify a couple dates to make it easy to verify we are getting expected results
--   you will likely need to modify the dates on lines 9 and 17 depending on when you running this
-- cold weather with precipitation should yield low visits
 UPDATE 
SNOWDEV.RAW.WEATHER_FORECAST SET 
MAX_TEMPERATURE_AIR_2M_F = 65,
AVG_TEMPERATURE_AIR_2M_F = 59,
TOT_PRECIPITATION_IN = 3.1
WHERE DATE_VALID_STD =   '2024-12-16' ;

-- precipitation has lesseend and it's getting a bit warmer
 UPDATE 
SNOWDEV.RAW.WEATHER_FORECAST SET 
MAX_TEMPERATURE_AIR_2M_F = 69,
AVG_TEMPERATURE_AIR_2M_F = 64,
TOT_PRECIPITATION_IN = 1.5
WHERE DATE_VALID_STD =   '2024-12-17' ;

**Generate Forecast Model**

In Snowsight - AI & ML Studio > Forecasting

Name = visit_traffic_forecasting_weather_data


1. XS_WAREHOUSE
2. Training data
    SNOWDEV.RAW.TRAFFIC_WEATHER_HISTORY
3. Target Column
    The column we want to predict > #visits
4. Select Timestamp column from training data
    DATE_VALID_STD (it's a date not timestamp)
5. Select a series identifier from your training data
   POSTAL_CODE
6. Use additional features 
- I selected ALL
- the gist is these are the columns will help with prediction
	MAX_TEMPERATURE_AIR_2M_F, 
	AVG_TEMPERATURE_AIR_2M_F, 
	TOTAL_PRECIPITATION_IN
7. Select your prediction data
    SNOWDEV.RAW.WEATHER_FORECAST
8. Select Timestamp column
    DATE_VALID_STD
9. Select a series Identifier
	Selected POSTAL_CODE
10. Configure predictions
	
    
Took defaults
Table name = My_forecasts_2024_12_09

A worksheet was generated, below is what was in the worksheet


**Model Setup**

In [ ]:
use role ACCOUNTADMIN;
--use warehouse XS_WAREHOUSE;
use warehouse LARGE_WAREHOUSE;
use database SNOWDEV;
use schema RAW;

In [ ]:
-- Inspect the first 10 rows of your training data. This is the data we'll use to create your model.
select * from TRAFFIC_WEATHER_HISTORY limit 10;

In [ ]:
-- Prepare your training data. Timestamp_ntz is a required format.
CREATE or replace VIEW TRAFFIC_WEATHER_HISTORY_v1 AS SELECT
    * EXCLUDE DATE_VALID_STD,
    to_timestamp_ntz(DATE_VALID_STD) as DATE_VALID_STD_v1
FROM TRAFFIC_WEATHER_HISTORY;

In [ ]:
-- Prepare your prediction data. Timestamp_ntz is a required format.
CREATE or replace VIEW WEATHER_FORECAST_v1 AS SELECT
    * EXCLUDE DATE_VALID_STD,
    to_timestamp_ntz(DATE_VALID_STD) as DATE_VALID_STD_v1
FROM WEATHER_FORECAST;

 
**Create Predictions**


In [ ]:
-- Create your model.
CREATE or replace SNOWFLAKE.ML.FORECAST visit_traffic_forecasting_weather_data(
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'TRAFFIC_WEATHER_HISTORY_v1'),
    SERIES_COLNAME => 'POSTAL_CODE',
    TIMESTAMP_COLNAME => 'DATE_VALID_STD_v1',
    TARGET_COLNAME => 'VISITS',
    CONFIG_OBJECT => { 'ON_ERROR': 'SKIP' }
);


In [ ]:
show snowflake.ml.forecast in database snowdev;



-- list models and version alternate approach
-- use database snowdev;
-- use schema raw;
-- SHOW SNOWFLAKE.ML.FORECAST; 

In [ ]:
  CREATE  OR REPLACE TABLE My_forecasts_2024_12_09 AS SELECT * FROM 
      TABLE(visit_traffic_forecasting_weather_data!FORECAST(
        INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'WEATHER_FORECAST_v1'),
        SERIES_COLNAME => 'POSTAL_CODE',
        TIMESTAMP_COLNAME => 'DATE_VALID_STD_v1',
        CONFIG_OBJECT => {'prediction_interval': 0.95}
    ));

In [ ]:
-- Unable to get this to run in notebook, need to investigate
-- Generate predictions and store the results to a table.
-- replaced by above CTAS approach

/*
BEGIN
    -- This is the step that creates your predictions.
    CALL visit_traffic_forecasting_weather_data!FORECAST(
        INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'WEATHER_FORECAST_v1'),
        SERIES_COLNAME => 'POSTAL_CODE',
        TIMESTAMP_COLNAME => 'DATE_VALID_STD_v1',
        -- Here we set your prediction interval.
        CONFIG_OBJECT => {'prediction_interval': 0.95}
    );
    -- These steps store your predictions to a table.
    LET x := SQLID;
    CREATE   TABLE My_forecasts_2024_12_09 AS SELECT * FROM TABLE(RESULT_SCAN(:x));
END;
*/

**Review Prediction Output**

In [ ]:
-- view predictions
-- NOTE 2024-12-16 has low forecast due to precipitation
-- NOTE 2024-12-17 has higher forecast than prior day, weathe is improving

SELECT * FROM My_forecasts_2024_12_09;


In [ ]:
use warehouse xs_warehouse;

In [ ]:
-- Review Historical and Prediction Data
--
SELECT POSTAL_CODE, DATE_VALID_STD, VISITS AS actual, NULL AS forecast, NULL AS lower_bound, NULL AS upper_bound
    FROM TRAFFIC_WEATHER_HISTORY
UNION ALL
SELECT replace(series, '"', '') as POSTAL_CODE, ts as DATE_VALID_STD, NULL AS actual, forecast, lower_bound, upper_bound
    FROM My_forecasts_2024_12_09
    ORDER BY DATE_VALID_STD DESC;

**Inspect Model Results**

In [ ]:
-- Inspect the accuracy metrics of your model. 
CALL visit_traffic_forecasting_weather_data!SHOW_EVALUATION_METRICS();

In [ ]:
-- Inspect the relative importance of your features, including auto-generated features. 
CALL visit_traffic_forecasting_weather_data!EXPLAIN_FEATURE_IMPORTANCE();